<img src="Maslow's_Hierarchy_of_Needs.svg.png" height = 600px>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import urllib.parse # combine URL components into URL string
import wikipediaapi # query wikipedia through api
import requests # standard for making HTTP requests

from statsmodels.tsa.seasonal import STL # seasonal decompositions
import statsmodels.tsa.stattools as smt


import pickle #  to serialize and deserialize objects in Python

from scipy import signal
import warnings
from urllib import request
from bs4 import BeautifulSoup
import json
import pytrends
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import wikipediaapi
import pprint as pp

In [5]:
# 
def create_dataframe(name_file):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file
    return: dataframe with the text file
    """
    df = pd.read_csv(name_file, header=None, names=['Topics'])
    df.drop_duplicates(inplace=True)
    return df

# Take only starting from the second word in each row
def remove_space(df):
    return df['Topics'].apply(lambda x: x.strip().replace(' ', '_'))

# Parse the topics into the URL format
def parse_topics_into_df(df, start_time, end_time):
    # change the spaces to underscores
    df['url'] = np.zeros(len(df))
    for index, row in df.iterrows():
        topic_value = row['Topics']
        df.loc[index, 'url'] = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/it.wikipedia.org/all-access/all-agents/{topic_value}/monthly/{start_time}/{end_time}'

    return df

# Create a new dataframe with timestamp from starting date to ending date
def create_dataframe_timestamp(starting_date, ending_date):
    df_timestamp = pd.DataFrame()
    df_timestamp['Timestamp'] = pd.date_range(start=starting_date, end=ending_date, freq='MS')
    return df_timestamp

# Define a function to fetch data from the URL and handle errors
def fetch_and_parse_url(url):
    try:
        request.urlopen(url).read()
        return True
    except request.HTTPError as e:
        if e.code == 404:
            return None  # or any other value or action you prefer for 404 errors
        else:
            return None  # or handle other HTTP errors as needed
    except Exception as e:
        
        return None  # or handle other exceptions as needed

def get_pageviews_wiki(url):
    """
    Gets the weekly pageviews for one Wikipedia page in one language in the desired period
    param: url: url of the Wikipedia page
    param: start_date: beginning of the desired period 
    param: end_date: end of the desired period 
    return: dataframe column with the monthly pageviews
    """
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    site_json=json.loads(soup.text)
    df=pd.DataFrame(site_json['items'])
    df=df['views']
    return df

def scrape_pageviews(df):
    pageview = pd.DataFrame()
    pageview['Timestamp'] = create_dataframe_timestamp('2019-10-01', '2023-02-01')['Timestamp']

    # Loop through the rows of the DataFrame and append the results of the function to the DataFrame
    for index, row in df.iterrows():
        url = row['url']
        if fetch_and_parse_url(url):
            pageview[row['Topics']] = get_pageviews_wiki(url)

    return pageview

# Specify the path to your text file
text_file_path = 'creativity.txt'

def create_dataframe_pageviews(name_file, start_time = '20191001', end_time = '20230213'):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file, start_time: beginning of the desired period, end_time: end of the desired period
    """
    df_topic = create_dataframe(name_file)
    df_topic['Topics'] = remove_space(df_topic)
    df_topic = parse_topics_into_df(df_topic, start_time, end_time)
    df_pageviews = scrape_pageviews(df_topic)
    return df_pageviews

# Read the text file into a DataFrame
df_pageviews = create_dataframe_pageviews(text_file_path)
print(type(df_pageviews))
print(df_pageviews.shape)
df_pageviews

<class 'pandas.core.frame.DataFrame'>
(41, 15)


,Timestamp,Innovation,Imagination,Design_Thinking,Collaboration,Curiosity,Inspiration,Storytelling,Performance_Art,Experiential_Learning,Avant-garde,Synthesis,Color_Theory,Open_Innovation,Gamification
0,2019-10-01,72,1842,1861,115.0,170,32,4128,218,72.0,442,72,43.0,17.0,2510
1,2019-11-01,112,1588,1868,21.0,241,54,4240,87,44.0,382,80,27.0,27.0,2590
2,2019-12-01,80,1059,1241,39.0,151,59,3817,31,16.0,379,75,51.0,27.0,2037
3,2020-01-01,95,1092,1807,29.0,197,35,5339,30,12.0,413,86,31.0,12.0,2281
4,2020-02-01,78,1086,1575,35.0,155,27,4843,20,5.0,377,57,23.0,18.0,1681
5,2020-03-01,82,1087,1744,37.0,328,17,4940,22,6.0,520,71,27.0,9.0,1870
6,2020-04-01,110,1517,2428,55.0,167,25,6371,36,17.0,573,92,24.0,29.0,2464
7,2020-05-01,106,1445,2628,27.0,193,25,5912,51,24.0,566,89,23.0,25.0,2470
8,2020-06-01,94,1183,2363,76.0,172,25,4764,30,14.0,456,68,16.0,21.0,2016
9,2020-07-01,84,1215,1532,54.0,201,35,3899,26,11.0,358,73,17.0,16.0,1721
